In [1]:
import spacy
import os
import pandas as pd

In [2]:
nlp = spacy.load("en_core_web_md")

In [3]:
term_df = pd.read_csv(r"D:\ulan_rel_0621\term.csv")
biography_df = pd.read_csv(r"D:\ulan_rel_0621\biography.csv")

In [4]:
term_df

,subject_id,term
0,500000131,Cesare Bernazzano
1,500000133,"Crabeth, Wouter Pietersz, II"
2,500000133,"Crabeth, Wouter Pietersz., II"
3,500000134,"McGlashan, Archibald A."
4,500000135,"Jadot, Jean Nicolas"
...,...,...
946962,500024225,"Juchymowytsch Rjepin, Illja"
946963,500359529,"Mössmer, Joseph"
946964,500196059,"Subirachs, Josep Maria"
946965,500255281,"Zbynek, Sekal"


In [5]:
biography_df

,birth_date,death_date,subject_id
0,1830,1892,500000934
1,1830,1892,500000934
2,1830,1892,500000934
3,1880,1980,500000935
4,1926,2026,500000936
...,...,...,...
546896,1930,2004,500086588
546897,1500,2080,500086588
546898,1871,2080,500407023
546899,1500,2080,500407023


In [61]:
"""
Time-Span

E67_Birth 
E69_Death
E12_Production
E8_Acquisition

"""
terms = list(term_df["term"])


In [231]:
def searchTerm(term, f):
    subject_id = 0
    if term.strip() in terms:
        subject_id = term_df[term_df["term"] == term.strip()]["subject_id"].values[0]

    if subject_id:
        return biography_df[biography_df["subject_id"] == subject_id][f].values[0]

    return -1

def searchBirth(time_col:pd.Series, person_col:pd.Series):
    
    df = pd.DataFrame({time_col.name:time_col, person_col.name:person_col})
    if len(df) > 100:
        df = df.sample(n=100, axis=0, random_state=1)
    df["search"] = df[person_col.name].apply(searchTerm, args=["birth_date"])
    df.drop_duplicates(inplace=True)
    df.drop(index=df[df["search"] == -1].index, inplace=True)
    res = list(df[time_col.name].values == df["search"])
    if res:
        return res.count(True) / len(res) > 0.8
    return False

def searchDeath(time_col:pd.Series, person_col:pd.Series):
    df = pd.DataFrame({time_col.name:time_col, person_col.name:person_col})
    if len(df) > 100:
        df = df.sample(n=100, axis=0, random_state=1)
    df["search"] = df[person_col.name].apply(searchTerm, args=["death_date"])
    df.drop_duplicates(inplace=True)
    df.drop(index=df[df["search"] == -1].index, inplace=True)
    res = list(df[time_col.name].values == df["search"])
    if res:
        return res.count(True) / len(res) > 0.8
    
    return False



def predTimeSpan(time_col:pd.Series, person_col:pd.Series):
    attr_name = time_col.name
    
    def seg(attr_name):
        s = ""
        if " " not in attr_name:
            for c in attr_name:
                if c.isupper():
                    s += " "
                if c.isalpha():
                    s += c
                else:
                    s += " "
            return s.strip()
        else:
            return attr_name
    
    def attr_similarity(attr_name, entity):
        
        return nlp(seg(attr_name)).similarity(nlp(entity))
    
    if attr_name.lower().find("birth") > -1 or searchBirth(time_col, person_col):
        return "E67_Birth"
    elif attr_name.lower().find("death") > -1 or searchDeath(time_col, person_col):
        return "E69_Death"
    elif attr_similarity(attr_name, "acquisition") > 0.6:
        return "E8_Acquisition"
    else:
        return "E12_Production"
    


In [184]:
os.chdir(r"D:\kg_20210906\sources-modified-20210828")
s01 = pd.read_csv('s01.csv')
s02 = pd.read_csv('s02.csv')
s03 = pd.read_csv('s03.csv')
s04 = pd.read_csv('s04.csv')
s05 = pd.read_csv('s05.csv')
s06 = pd.read_csv('s06.csv', encoding='ansi')
s07 = pd.read_csv('s07.csv')
s08 = pd.read_csv('s08.csv', encoding='ansi')
s09 = pd.read_csv('s09.csv')
s10 = pd.read_csv('s10.csv')
s11 = pd.read_csv('s11.csv')
s12 = pd.read_csv('s12.csv', encoding='ansi')
s13 = pd.read_csv('s13.csv', encoding='ansi')
s14 = pd.read_csv('s14.csv')
s15 = pd.read_csv('s15.csv')
s16 = pd.read_csv('s16.csv')
s17 = pd.read_csv('s17.csv')
s18 = pd.read_csv('s18.csv')
s19 = pd.read_csv('s19.csv')
s20 = pd.read_csv('s20.csv')
s21 = pd.read_csv('s21.csv', encoding='ansi')
s22 = pd.read_csv('s22.csv')
s23 = pd.read_csv('s23.csv', encoding='ansi')
s24 = pd.read_csv('s24.csv')
s25 = pd.read_csv('s25.csv')
s26 = pd.read_csv('s26.csv')
s28 = pd.read_csv('s28.csv')
s29 = pd.read_csv('s29.csv')

In [232]:
result_df = pd.DataFrame(columns = ["ds","col_name", "correct_type", "pred_type"])



# while True:
        
#     arg_in = input()
#     if arg_in == "exit":
#         break
    
#     args = arg_in.split(",")
#     result_df.append({"ds":args[0], "col_name":args[1], "correct_type":args[2], 
#                               "pred_type":predTimeSpan(eval(args[0])[args[1]], eval(args[0])[args[3]])}, ignore_index=True)
    
args_df = pd.read_excel("args.xlsx", header=None)

# result_df = result_df.append({"ds":"s01", "col_name":"Begin Date", "correct_type":"E67_Birth", 
#                           "pred_type":predTimeSpan(s01["Begin Date"], s01["Attribution"])}, ignore_index=True)

for i in range(len(args_df)):
    args = list(args_df.loc[i,])
    print(args)
    result_df = result_df.append({"ds":args[0], 
                                  "col_name":args[1], 
                                  "correct_type":args[2], 
                                  "pred_type":predTimeSpan(eval(args[0])[args[1]], eval(args[0])[args[3]])},
                                  ignore_index=True)


['s01', 'Begin Date', 'E67_Birth', 'Attribution']
['s01', 'End Date', 'E69_Death', 'Attribution']
['s01', 'Dated', 'E12_Produciton', 'Attribution']
['s02', 'Artist Birth Date', 'E67_Birth', 'Artist Name']
['s02', 'Artist Death Date', 'E69_Death', 'Artist Name']
['s02', 'Object Earliest Date', 'E12_Produciton', 'Artist Name']
['s02', 'Object Latest Date', 'E12_Produciton', 'Artist Name']
['s03', 'birthDate', 'E67_Birth', 'name']
['s03', 'deathDate', 'E69_Death', 'name']
['s04', 'birthDate', 'E67_Birth', 'artist']
['s04', 'deathDate', 'E69_Death', 'artist']
['s04', 'creationDate', 'E12_Produciton', 'artist']
['s05', 'Date', 'E12_Produciton', 'Who']
['s06', 'ArtistBornDate', 'E67_Birth', 'Artist']
['s06', 'ArtistDiedDate', 'E69_Death', 'Artist']
['s06', 'AquisitionDate', 'E8_Acquisiton', 'Artist']
['s06', 'DateOfWork', 'E12_Produciton', 'Artist']
['s07', 'birth_year', 'E67_Birth', 'name']
['s07', 'death_year', 'E69_Death', 'name']
['s07', 'dates', 'E12_Produciton', 'name']
['s09', 'birthD

C:\Users\LRSLXD~1\AppData\Local\Temp/ipykernel_13476/4220642074.py:58: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return nlp(seg(attr_name)).similarity(nlp(entity))


['s18', 'birthDate', 'E67_Birth', 'name']
['s18', 'deathDate', 'E69_Death', 'name']
['s19', 'birthDate', 'E67_Birth', 'artist']
['s19', 'deathDate', 'E69_Death', 'artist']
['s19', 'creationDate', 'E12_Produciton', 'artist']
['s20', 'birth_date', 'E67_Birth', 'Artist_Name']
['s20', 'death_date', 'E69_Death', 'Artist_Name']
['s20', 'Production_Year', 'E12_Produciton', 'Artist_Name']
['s21', 'birthDate', 'E67_Birth', 'artistName']
['s21', 'deathDate', 'E69_Death', 'artistName']
['s21', 'dateOfStartOfCreation', 'E12_Produciton', 'artistName']
['s21', 'dateOfEndOfCreation', 'E12_Produciton', 'artistName']
['s22', 'Artwork_Year', 'E12_Produciton', 'Artist_Name']
['s23', 'bornDate', 'E67_Birth', 'Artist']
['s23', 'deathDate', 'E69_Death', 'Artist']
['s23', 'AquisitionDate', 'E8_Acquisiton', 'Artist']
['s23', 'DateOfWork', 'E12_Produciton', 'Artist']
['s24', 'birth_date', 'E67_Birth', 'artist']
['s24', 'death_date', 'E69_Death', 'artist']
['s24', 'made', 'E12_Produciton', 'artist']
['s25', 'Bi

In [220]:
result_df

,ds,col_name,correct_type,pred_type
0,s01,Begin Date,E67_Birth,E67_Birth
1,s01,End Date,E69_Death,E69_Death
2,s01,Dated,E12_Produciton,E12_Production
3,s02,Artist Birth Date,E67_Birth,E67_Birth
4,s02,Artist Death Date,E69_Death,E69_Death
...,...,...,...,...
56,s26,deathYear,E69_Death,E69_Death
57,s26,when-acquired,E8_Acquisiton,E8_Acquisition
58,s28,Maker Birth Date,E67_Birth,E67_Birth
59,s28,Maker Death Date,E69_Death,E69_Death


In [233]:
result_df.to_csv("result.csv", index=False)